In [1]:
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv('insurance.csv')
df.head()

/var/folders/n0/fpj669xs7mdf7dy6vwdnptk40000gn/T/ipykernel_58468/3557345218.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [2]:
df.corr(numeric_only =True)

,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


In [17]:
# uppercase makes it a matrix :) that's why the Xs are upper and y is lower
y = df['charges']
X = df[['age', 'bmi', 'children']] # Manual way of getting only the features we want
X = df.select_dtypes(include='number').assign(const=1) #this is required to make it work... idk why
X.drop(columns=['charges'], inplace=True)
# instead of the one above I can also do: X['const'] = 1 
X.head()

,age,bmi,children,const
0,19,27.900,0,1
1,18,33.770,1,1
2,28,33.000,3,1
3,33,22.705,0,1
4,32,28.880,0,1


In [18]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     60.69
Date:                Tue, 30 Jan 2024   Prob (F-statistic):           8.80e-37
Time:                        14:45:28   Log-Likelihood:                -14392.
No. Observations:                1338   AIC:                         2.879e+04
Df Residuals:                    1334   BIC:                         2.881e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age          239.9945     22.289     10.767      0.0

- R square is the model fit (combined affect size)
- Coefficient (coef) shows the affect size separately
- t and p show "reliability"
    - if the p is really low, I would expect to get this same coefficient next time I do this
    - t ranges from -infinity and +infinity and is the same sign(-/+) as the coefficient
    - 1.692 t is the same as .05 p
    - p value is very affected by sample size
    - const coefficient is the y-value

In [19]:
results.predict([44,18,3,1])

array([11249.60803894])

add text features below:
    - smoker is easy to turn yes/no into 1/0

In [29]:
df = pd.read_csv('insurance.csv')
# df = pd.get_dummies(df, columns=['sex', 'smoker', 'region']) #manual
df = pd.get_dummies(df, columns=df.select_dtypes(['object']).columns, drop_first=True) #dynamic
df =df.astype('int64')
df.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27,0,16884,0,1,0,0,1
1,18,33,1,1725,1,0,0,1,0
2,28,33,3,4449,1,0,0,1,0
3,33,22,0,21984,1,0,1,0,0
4,32,28,0,3866,1,0,1,0,0


In [30]:
y = df['charges']
X = df.drop(columns=['charges'])
print(sm.OLS(y, X.assign(const=1)).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.7
Date:                Tue, 30 Jan 2024   Prob (F-statistic):               0.00
Time:                        15:15:04   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                256.9773     11.898  